In [11]:
import sys
sys.path.append("../src")

In [12]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
%autoreload 2

import sys
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torchvision import transforms, datasets

import matplotlib.pyplot as plt
from pytorch_impl.nns import ResNet, CNN
from pytorch_impl.nns import warm_up_batch_norm
from pytorch_impl.estimators import ForrestEstimator
from pytorch_impl import ClassifierTraining

In [14]:
num_classes = 10

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

cifar10_stats = {
    "mean" : (0.4914, 0.4822, 0.4465),
    "std"  : (0.24705882352941178, 0.24352941176470588, 0.2615686274509804),
}

transform_train = transforms.Compose([
    transforms.Lambda(lambda x: np.asarray(x)),
    transforms.Lambda(lambda x: np.pad(x, [(4, 4), (4, 4), (0, 0)], mode='reflect')),
    transforms.Lambda(lambda x: Image.fromarray(x)),
    transforms.RandomCrop(32),
    
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cifar10_stats['mean'], cifar10_stats['std']),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar10_stats['mean'], cifar10_stats['std']),
])

train_loader = torch.utils.data.DataLoader(
                  datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train),
               batch_size=128, shuffle=True, pin_memory=True)

test_loader  = torch.utils.data.DataLoader(
                  datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test),
               batch_size=128, shuffle=True, pin_memory=True)

device

Files already downloaded and verified
Files already downloaded and verified


device(type='cuda', index=0)

In [15]:
model = CNN(num_classes=1, input_channels=3, num_channels=32).to(device)
warm_up_batch_norm(model, train_loader, device, limit=2)

eval BatchNorm2d
eval BatchNorm2d
eval BatchNorm2d
eval BatchNorm2d


CNN(
  (layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
  )
  (classifier): Linear(in_features=32, out_features=1, bias=False)
)

In [16]:
train_loader = torch.utils.data.DataLoader(
                  datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train),
               batch_size=1024, shuffle=True, pin_memory=True)

test_loader  = torch.utils.data.DataLoader(
                  datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test),
               batch_size=1024, shuffle=True, pin_memory=True)

device

Files already downloaded and verified
Files already downloaded and verified


device(type='cuda', index=0)

In [17]:
_, (X, y) = next(enumerate(train_loader))
X, y = X.to(device), y.to(device)

In [ ]:
forrest_estimator = ForrestEstimator(model, input_channels=3, device=device)

In [ ]:
forrest_estimator.fit_batch(X, y)

train_size (65536, 54)


In [ ]:
_, (X, y) = next(enumerate(train_loader))
X, y = X.to(device), y.to(device)

y_pred = forrest_estimator.predict(X)
(y_pred.argmax(dim=1) == y).float().mean()